In [1]:
import dace
import cupy as cp
import numpy as np
from IPython.display import Code
from typing import Optional

from dace import SDFG, properties
from dace.config import Config
from dace.transformation import pass_pipeline as ppl, transformation
from dace.sdfg import nodes
from dace import dtypes
from dace.transformation.passes.shared_memory_synchronization import DefaultSharedMemorySync
from dace.sdfg.state import LoopRegion, ConditionalBlock


In [2]:

# SDFG and the main state
sdfg = dace.SDFG("asyn_cpy_sdfg")
state = sdfg.add_state("main")

# Arrays and access nodes
sdfg.add_array("A", (128,), dace.uint32, storage=dace.dtypes.StorageType.GPU_Global)
sdfg.add_array("B", (128,), dace.uint32, storage=dace.dtypes.StorageType.GPU_Global)
sdfg.add_array("S", (128,), dace.uint32, storage=dace.dtypes.StorageType.GPU_Shared, transient=True)

a_acc = state.add_read("A")
b_acc = state.add_access("B")
s_acc = state.add_access("S")




# Device and thread-block maps
gpu_map_entry, gpu_map_exit = state.add_map("gpu_map", dict(bid="0:128:128"), schedule=dace.dtypes.ScheduleType.GPU_Device)
tb_map_entry, tb_map_exit = state.add_map("tb_map", dict(tid="0:128"), schedule=dace.dtypes.ScheduleType.GPU_ThreadBlock)

# Edges with proper data flow
# Global to device scope
state.add_edge(a_acc, None, gpu_map_entry, None, dace.Memlet("A[0:128]"))
# Device scope to thread-block scope
state.add_edge(gpu_map_entry, None, s_acc, None,  dace.Memlet("A[0:128]->S[0:128]"))
state.add_edge(s_acc, None,  tb_map_entry, None, dace.Memlet("S[0:128]"))

assign_tasklet = state.add_tasklet(
    "assign", inputs={"__in_S"}, outputs={"__out_S"},
    code="__out_S = __in_S;",
    language=dace.dtypes.Language.CPP
)


state.add_edge(tb_map_entry, None, assign_tasklet, "__in_S", dace.Memlet("S[tid]"))
state.add_edge(assign_tasklet, "__out_S", tb_map_exit, None, dace.Memlet("B[tid]"))
state.add_edge(tb_map_exit, None, gpu_map_exit, None, dace.Memlet("B[0:128]"))
state.add_edge(gpu_map_exit, None, b_acc, None, dace.Memlet("B[0:128]"))


# pipeline and async related 

pipeline_name = "pipeline"
s_acc.async_copy = True
s_acc.async_pipeline = pipeline_name
sdfg.metadata = {
    s_acc.guid: {
        "pipelines": {
            pipeline_name: {
                "pipeline_depth" : 1
            }
        }
    }
}


acquire_pipeline_tasklet = state.add_tasklet(
    "acquire", inputs={}, outputs={},
    code=f"{pipeline_name}.producer_acquire();",
    language=dace.dtypes.Language.CPP
)

commit_pipeline_tasklet = state.add_tasklet(
    "commit", inputs={}, outputs={},
    code=f"{pipeline_name}.producer_commit();",
    language=dace.dtypes.Language.CPP
)

wait_pipeline_tasklet = state.add_tasklet(
    "wait", inputs={}, outputs={},
    code=f"{pipeline_name}.consumer_wait();",
    language=dace.dtypes.Language.CPP
)

release_pipeline_tasklet = state.add_tasklet(
    "release", inputs={}, outputs={},
    code=f"{pipeline_name}.consumer_release();",
    language=dace.dtypes.Language.CPP
)



state.add_edge(gpu_map_entry, None, acquire_pipeline_tasklet, None, dace.Memlet())
state.add_edge(acquire_pipeline_tasklet, None, s_acc, None, dace.Memlet())

state.add_edge(s_acc, None, commit_pipeline_tasklet, None, dace.Memlet())
state.add_edge(commit_pipeline_tasklet, None, wait_pipeline_tasklet, None, dace.Memlet())
state.add_edge(wait_pipeline_tasklet, None, tb_map_entry, None, dace.Memlet())

state.add_edge(tb_map_exit, None, release_pipeline_tasklet, None, dace.Memlet())
state.add_edge(release_pipeline_tasklet, None, gpu_map_exit, None, dace.Memlet())




# Fill scope connectors
state.fill_scope_connectors()


# Display the SDFG
sdfg

SDFG (asyn_cpy_sdfg)

In [3]:

Code(sdfg.generate_code()[1].clean_code)

#include <cuda_runtime.h>
#include <dace/dace.h>

// New, cooperative groups and asnyc copy
#include <cooperative_groups/memcpy_async.h>
#include <cuda/pipeline>

namespace cg = cooperative_groups;


struct asyn_cpy_sdfg_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_experimental_cuda(asyn_cpy_sdfg_state_t *__state);
DACE_EXPORTED int __dace_exit_experimental_cuda(asyn_cpy_sdfg_state_t *__state);



int __dace_init_experimental_cuda(asyn_cpy_sdfg_state_t *__state) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(0, 0);

    // Create cuda streams and events
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_experimental_cuda(asyn_cpy_sdfg_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 0; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(asyn_cpy_sdfg_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 0)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(asyn_cpy_sdfg_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 0; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void __launch_bounds__(128) gpu_map_0_0_3(dace::uint * __restrict__ A, dace::uint * __restrict__ B)
{
    __shared__ dace::uint S[128];

    auto block = cg::this_thread_block();

    const uint pipeline_depth_pipeline = 1;
    __shared__ cuda::pipeline_shared_state<cuda::thread_scope::thread_scope_block, pipeline_depth_pipeline> shared_state_pipeline;
    auto pipeline = cuda::make_pipeline(block, &shared_state_pipeline);

    int bid = (128 * blockIdx.x);
    {

        ///////////////////
        pipeline.producer_acquire();
        ///////////////////

    }
    cuda::memcpy_async(block, S, A, 128 *sizeof(dace::uint), pipeline);
    {

        ///////////////////
        pipeline.producer_commit();
        ///////////////////

    }
    {

        ///////////////////
        pipeline.consumer_wait();
        ///////////////////

    }
    {
        int tid = threadIdx.x;
        {
            dace::uint __in_S = S[tid];
            dace::uint __out_S;

            ///////////////////
            __out_S = __in_S;
            ///////////////////

            B[tid] = __out_S;
        }
    }
    {

        ///////////////////
        pipeline.consumer_release();
        ///////////////////

    }
}


DACE_EXPORTED void __dace_runkernel_gpu_map_0_0_3(asyn_cpy_sdfg_state_t *__state, dace::uint * __restrict__ A, dace::uint * __restrict__ B);
voi

In [4]:

A = cp.ones((128,), dtype=cp.uint32)
B = cp.zeros((128,), dtype=cp.uint32)

print(f"A before:\n{A}")
print(f"B before:\n{B}")

sdfg(A=A, B=B)

print(f"A after:\n{A}")
print(f"B after:\n{B}")


A before:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
B before:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
A after:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
B after:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 